In [4]:
!pip install transformers




[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import requests

api_key = "Your_token_goes_here"
company = "AAPL"

api1 = f"https://financialmodelingprep.com/api/v3/quote/{company}?apikey={api_key}"
api2 = f"https://financialmodelingprep.com/api/v3/otc/real-time-price/{company}?apikey={api_key}"
api3 = f"https://financialmodelingprep.com/api/v3/stock-price-change/{company}?apikey={api_key}"
api4 = f"https://financialmodelingprep.com/api/v3/stock/full/real-time-price/{company}?apikey={api_key}"
api5 = f"https://financialmodelingprep.com/api/v3/income-statement/{company}?period=annual&apikey={api_key}"
api6 = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{company}?period=annual&apikey={api_key}"
api7 = f"https://financialmodelingprep.com/api/v3/income-statement-as-reported/{company}?period=annual&limit=50&apikey={api_key}"

In [7]:
api_response_1 = requests.get(api1).json()
print(api_response_1)

[{'symbol': 'AAPL', 'name': 'Apple Inc.', 'price': 223.83, 'changesPercentage': 0.5345, 'change': 1.19, 'dayLow': 219.8, 'dayHigh': 223.98, 'yearHigh': 260.1, 'yearLow': 164.08, 'marketCap': 3365933157000, 'priceAvg50': 239.2804, 'priceAvg200': 217.2806, 'exchange': 'NASDAQ', 'volume': 62954541, 'avgVolume': 47048363, 'open': 219.94, 'previousClose': 222.64, 'eps': 6.75, 'pe': 33.16, 'earningsAnnouncement': '2025-01-30T21:00:00.000+0000', 'sharesOutstanding': 15037900000, 'timestamp': 1737579601}]


In [8]:
api_5_data = requests.get(api5).json()
api_6_data = requests.get(api6).json()
api_7_data = requests.get(api7).json()

In [9]:
def build_historical_data(api_5, api_6, api_7):
    historical_data = []

    for record_5 in api_5:
        date = record_5["date"]
        record_6 = next((r for r in api_6 if r["date"] == date), {})
        record_7 = next((r for r in api_7 if r["date"] == date), {})

        combined_record = {
            "date": date,
            "revenue": record_5.get("revenue"), 
            "netIncome": record_5.get("netIncome") or record_6.get("netIncome"),  
            "eps": record_5.get("eps") or record_7.get("earningspersharebasic"),  
            "freeCashFlow": record_6.get("freeCashFlow"), 
            "operatingIncome": record_5.get("operatingIncome") or record_7.get("operatingincomeloss"), 
            "grossProfit": record_7.get("grossprofit"),  
        }

        historical_data.append(combined_record)

    return historical_data

In [10]:
historical_data = build_historical_data(api_5_data, api_6_data, api_7_data)

for record in historical_data:
    print(record)

{'date': '2024-09-28', 'revenue': 391035000000, 'netIncome': 93736000000, 'eps': 6.11, 'freeCashFlow': 108807000000, 'operatingIncome': 123216000000, 'grossProfit': None}
{'date': '2023-09-30', 'revenue': 383285000000, 'netIncome': 96995000000, 'eps': 6.16, 'freeCashFlow': 99584000000, 'operatingIncome': 114301000000, 'grossProfit': None}
{'date': '2022-09-24', 'revenue': 394328000000, 'netIncome': 99803000000, 'eps': 6.15, 'freeCashFlow': 111443000000, 'operatingIncome': 119437000000, 'grossProfit': 170782000000}
{'date': '2021-09-25', 'revenue': 365817000000, 'netIncome': 94680000000, 'eps': 5.67, 'freeCashFlow': 92953000000, 'operatingIncome': 108949000000, 'grossProfit': 152836000000}
{'date': '2020-09-26', 'revenue': 274515000000, 'netIncome': 57411000000, 'eps': 3.31, 'freeCashFlow': 73365000000, 'operatingIncome': 66288000000, 'grossProfit': 104956000000}


In [12]:
current_metrics = {
    "price": api_response_1[0]["price"],
    "pe_ratio": api_response_1[0]["pe"],
    "eps": api_response_1[0]["eps"],
    "year_high": api_response_1[0]["yearHigh"],
    "year_low": api_response_1[0]["yearLow"],
    "price_avg_50": api_response_1[0]["priceAvg50"],
    "price_avg_200": api_response_1[0]["priceAvg200"],
    "volume": api_response_1[0]["volume"],
    "avg_volume": api_response_1[0]["avgVolume"],
}
print(current_metrics)

{'price': 223.83, 'pe_ratio': 33.16, 'eps': 6.75, 'year_high': 260.1, 'year_low': 164.08, 'price_avg_50': 239.2804, 'price_avg_200': 217.2806, 'volume': 62954541, 'avg_volume': 47048363}


In [13]:
def calculate_growth(data, key):
    growth_rates = []
    for i in range(1, len(data)):
        growth = (data[i - 1][key] - data[i][key]) / data[i][key]
        growth_rates.append(growth)
    
    return growth_rates[0] if growth_rates else None

In [14]:
historical_trends = {
    "revenue_growth": calculate_growth(historical_data, key="revenue"),
    "net_income_growth": calculate_growth(historical_data, key="netIncome"),
    "eps_growth": calculate_growth(historical_data, key="eps"),
    "free_cash_flow_growth": calculate_growth(historical_data, key="freeCashFlow"),
}

In [15]:
def calculate_score(current_metrics, historical_trends):
    # Assign weights to different factors
    weights = {
        "pe_ratio": -0.2,   # Lower P/E ratio is better
        "eps_growth": 0.3,   # Higher EPS growth is better
        "revenue_growth": 0.2,   # Higher revenue growth is better
        "free_cash_flow_growth": 0.2,   # Higher free cash flow growth is better
        "price_vs_50_avg": 0.1,   # Price closer to the 50-day average is better
        "price_vs_200_avg": 0.1   # Price above the 200-day average is better
    }

    # Normalize and calculate individual scores
    pe_score = max(0, min(1, (20 / current_metrics["pe_ratio"])))   # Normalize P/E ratio (ideal <20)
    eps_growth_score = max(0, min(1, historical_trends["eps_growth"]))
    revenue_growth_score = max(0, min(1, historical_trends["revenue_growth"]))
    free_cash_flow_score = max(0, min(1, historical_trends["free_cash_flow_growth"]))
    
    price_vs_50_avg_score = max(0, min(1, (1 - abs(current_metrics["price"] - current_metrics["price_avg_50"]) / current_metrics["price_avg_50"])))
    
    price_vs_200_avg_score = max(0, min(1, (1 - abs(current_metrics["price"] - current_metrics["price_avg_200"]) / current_metrics["price_avg_200"])))

    # Weighted sum of scores
    score = (
        weights["pe_ratio"] * pe_score +
        weights["eps_growth"] * eps_growth_score +
        weights["revenue_growth"] * revenue_growth_score +
        weights["free_cash_flow_growth"] * free_cash_flow_score +
        weights["price_vs_50_avg"] * price_vs_50_avg_score +
        weights["price_vs_200_avg"] * price_vs_200_avg_score
    )

    return max(0, min(1, score))   # Ensure score is between [0 and 1]

score = calculate_score(current_metrics, historical_trends)

In [19]:
def provide_reasoning(score, current_metrics, historical_trends):
    reasoning = []

    # P/E Ratio Analysis
    if current_metrics["pe_ratio"] > 30:
        reasoning.append("The P/E ratio is high at {:.2f}, indicating potential overvaluation.".format(current_metrics["pe_ratio"]))
    else:
        reasoning.append("The P/E ratio is reasonable at {:.2f}.".format(current_metrics["pe_ratio"]))

    # EPS Growth Analysis
    if historical_trends["eps_growth"] < 0:
        reasoning.append("EPS has declined by {:.2%}, indicating weaker profitability.".format(historical_trends["eps_growth"]))
    else:
        reasoning.append("EPS has grown by {:.2%}, indicating stronger profitability.".format(historical_trends["eps_growth"]))

    # Revenue Growth Analysis
    if historical_trends["revenue_growth"] < 0:
        reasoning.append("Revenue has declined by {:.2%}, which may indicate slowing business growth.".format(historical_trends["revenue_growth"]))
    else:
        reasoning.append("Revenue has grown by {:.2%}, showing positive business performance.".format(historical_trends["revenue_growth"]))

    # Free Cash Flow Analysis
    if historical_trends["free_cash_flow_growth"] > 0:
        reasoning.append("Free cash flow has increased by {:.2%}, indicating strong financial health.".format(historical_trends["free_cash_flow_growth"]))
    else:
        reasoning.append("Free cash flow has declined by {:.2%}, which may indicate weaker financial stability.".format(historical_trends["free_cash_flow_growth"]))

    return "\n".join(reasoning)

# Example usage
reasoning = provide_reasoning(score, current_metrics=current_metrics, historical_trends=historical_trends)
print(reasoning)


The P/E ratio is high at 33.16, indicating potential overvaluation.
EPS has declined by -0.81%, indicating weaker profitability.
Revenue has grown by 2.02%, showing positive business performance.
Free cash flow has increased by 9.26%, indicating strong financial health.


In [17]:
score

0.09246849633931024

In [22]:
def calculate_score_and_decision(current_metrics, historical_trends):

    # Step 1: Assign weights to different factors
    weights = {
        "pe_ratio": -0.2,  # Lower P/E ratio is better
        "eps_growth": 0.3,  # Higher EPS growth is better
        "revenue_growth": 0.2,  # Higher revenue growth is better
        "free_cash_flow_growth": 0.2,  # Higher free cash flow growth is better
        "price_vs_50_avg": 0.1,  # Price closer to the 50-day average is better
        "price_vs_200_avg": 0.1,  # Price above the 200-day average is better
    }

    # Step 2: Normalize individual scores for each factor
    pe_score = max(0, min(1, (20 / current_metrics["pe_ratio"])))  # Normalize P/E ratio (ideal <20)
    eps_growth_score = max(0, min(1, historical_trends["eps_growth"]))  # EPS growth rate
    revenue_growth_score = max(0, min(1, historical_trends["revenue_growth"]))  # Revenue growth rate
    free_cash_flow_score = max(0, min(1, historical_trends["free_cash_flow_growth"]))  # Free cash flow growth

    price_vs_50_avg_score = max(
        0,
        min(
            1,
            (1 - abs(current_metrics["price"] - current_metrics["price_avg_50"]) / current_metrics["price_avg_50"]),
        ),
    )  # Price vs. 50-day moving average

    price_vs_200_avg_score = max(
        0,
        min(
            1,
            (1 - abs(current_metrics["price"] - current_metrics["price_avg_200"]) / current_metrics["price_avg_200"]),
        ),
    )  # Price vs. 200-day moving average

    # Step 3: Calculate weighted score
    score = (
        weights["pe_ratio"] * pe_score +
        weights["eps_growth"] * eps_growth_score +
        weights["revenue_growth"] * revenue_growth_score +
        weights["free_cash_flow_growth"] * free_cash_flow_score +
        weights["price_vs_50_avg"] * price_vs_50_avg_score +
        weights["price_vs_200_avg"] * price_vs_200_avg_score
    )

    # Ensure the score is between [0 and 1]
    final_score = max(0, min(1, score))

    # Step 4: Make a decision based on the score
    decision = "Buy" if final_score > 0.5 else "Sell"

    # Step 5: Provide reasoning for the decision
    reasoning = []
    
    if pe_score < 0.5:
        reasoning.append(f"The P/E ratio of {current_metrics['pe_ratio']:.2f} suggests overvaluation.")
    else:
        reasoning.append(f"The P/E ratio of {current_metrics['pe_ratio']:.2f} is reasonable.")

    if eps_growth_score > 0.5:
        reasoning.append(f"EPS growth of {historical_trends['eps_growth']:.2%} indicates strong profitability.")
    else:
        reasoning.append(f"EPS growth of {historical_trends['eps_growth']:.2%} is weak.")

    if revenue_growth_score > 0.5:
        reasoning.append(f"Revenue growth of {historical_trends['revenue_growth']:.2%} shows positive business performance.")
    else:
        reasoning.append(f"Revenue growth of {historical_trends['revenue_growth']:.2%} is below expectations.")

    if free_cash_flow_score > 0.5:
        reasoning.append(f"Free cash flow growth of {historical_trends['free_cash_flow_growth']:.2%} indicates strong financial health.")
    else:
        reasoning.append(f"Free cash flow growth of {historical_trends['free_cash_flow_growth']:.2%} is weak.")

    if price_vs_50_avg_score > 0.5:
        reasoning.append("The stock price is close to its short-term (50-day) moving average.")
    else:
        reasoning.append("The stock price is far from its short-term (50-day) moving average.")

    if price_vs_200_avg_score > 0.5:
        reasoning.append("The stock price is above its long-term (200-day) moving average.")
    else:
        reasoning.append("The stock price is below its long-term (200-day) moving average.")

    return {
        "score": round(final_score, 4),
        "decision": decision,
        "reasoning": "\n".join(reasoning),
    }


result = calculate_score_and_decision(current_metrics, historical_trends)

print(f"Score: {result['score']}")
print(f"Decision: {result['decision']}")
print("Reasoning:")
print(result['reasoning'])


Score: 0.0925
Decision: Sell
Reasoning:
The P/E ratio of 33.16 is reasonable.
EPS growth of -0.81% is weak.
Revenue growth of 2.02% is below expectations.
Free cash flow growth of 9.26% is weak.
The stock price is close to its short-term (50-day) moving average.
The stock price is above its long-term (200-day) moving average.
